In [6]:
%pip install flask-cors

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import base64
from PIL import Image
import io
import numpy as np

In [2]:
# data import
dataset = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data()

In [3]:
# data normalization
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [6]:
# model config 784-50-100-500-1000-10-10
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(784, activation = 'relu'))
model.add(tf.keras.layers.Dense(50, activation = 'relu'))
model.add(tf.keras.layers.Dense(100, activation = 'relu'))
model.add(tf.keras.layers.Dense(500, activation = 'relu'))
model.add(tf.keras.layers.Dense(1000, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [8]:
# fit model
model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0715 - accuracy: 0.9804
Epoch 2/3
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0570 - accuracy: 0.9841
Epoch 3/3
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0491 - accuracy: 0.9863


In [9]:
# test
loss, accuracy = model.evaluate(x_test, y_test)
print('accuracy =', accuracy, 'loss =', loss)

313/313 [==============================] - 2s 5ms/step - loss: 0.1038 - accuracy: 0.9752
accuracy = 0.9752000570297241 loss = 0.10379461199045181


In [10]:
# save model
model.save('digits_recognition')

INFO:tensorflow:Assets written to: digits_recognition/assets


In [ ]:
import tensorflow as tf
import base64
from PIL import Image
import io
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin

model = tf.keras.models.load_model('./digits_recognition')

def base64_to_array(base64_str):
    stripped = base64_str.split(',')[1]
    stripped = stripped.replace(" ", "+") # fix '+' being as a space
    image_bytes = base64.b64decode(stripped)
    image = Image.open(io.BytesIO(image_bytes))
    image_array = np.array(image);
    flattened_array = image_array[:,:,0]
    normalized_array = flattened_array / 255
    
    return normalized_array

def get_predicted_number(image_array):
    batch = np.array([image_array])
    prediction = model.predict(batch)
    
    return [np.argmax(prediction[0]), np.max(prediction[0])]

app = Flask(__name__, static_folder='frontend')

@app.route('/classify_number', methods=['POST'])
@cross_origin()
def echo():
    image_base64 = request.json.get('image')
    response = jsonify({'prediction': str(-1), 'probability': str(-1)})
    if (image_base64):
        image_array = base64_to_array(image_base64)
        result = get_predicted_number(image_array)
        response = jsonify({'prediction': str(result[0]), 'probability': str(result[1])})
    
    return response

@app.route('/', defaults={'path': ''})
@app.route('/<path:path>')
def serve(path):
    if path != "" and os.path.exists(app.static_folder + '/' + path):
        return send_from_directory(app.static_folder, path)
    else:
        return send_from_directory(app.static_folder, 'index.html')

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Oct/2023 03:48:14] "OPTIONS /classify_number HTTP/1.1" 200 -


1/1 [==============================] - 0s 151ms/step


127.0.0.1 - - [08/Oct/2023 03:48:14] "POST /classify_number HTTP/1.1" 200 -


1/1 [==============================] - 0s 19ms/step


...

127.0.0.1 - - [08/Oct/2023 03:54:08] "POST /classify_number HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2023 03:54:17] "OPTIONS /classify_number HTTP/1.1" 200 -


1/1 [==============================] - 0s 13ms/step


127.0.0.1 - - [08/Oct/2023 03:54:17] "POST /classify_number HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2023 03:54:40] "OPTIONS /classify_number HTTP/1.1" 200 -


1/1 [==============================] - 0s 14ms/step


127.0.0.1 - - [08/Oct/2023 03:54:40] "POST /classify_number HTTP/1.1" 200 -


In [192]:

    
print(get_predicted_number([base64_to_array("data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAYAAAByDd+UAAAAAXNSR0IArs4c6QAAAMNJREFUSEvdllEKgCAQRNeb6cnUk6knM0ZYiCKwkRVKCPoI3+444+ZEpMvG5VaA3nvBk1KaLpkGAlRKkVqrhBDsgb33AcMyBaKzGKO01gYQ72ZAlTHnPM5N4SbAKwxSqllMTAODQMbz5mbAJ+lQBORV88xYlY4FNt8ORDScQ83zi+6QcSjK+g6QOb+lDrcDGcPQHbKG+Q6QudI0qVQsWMPQkv4fyEaCklQH8dtLmzbNSgbpDt/+OJ2HFxWL+el3/3I78ABK5J0B3Y5xzwAAAABJRU5ErkJggg==")]))

ValueError: in user code:

    File "/Users/arturpietrzak/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/arturpietrzak/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/arturpietrzak/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/arturpietrzak/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/arturpietrzak/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [191]:
model.predict(x_train)

1875/1875 [==============================] - 3s 2ms/step


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)